# Overview and Notes

What are interesting scenarios for exploring correlations?
* Expression-dependency correlations for dependency genes that are druggable. For these, the expression correlates are informative for selection of sensitive cell lines/patients. In addition, causally related expression correlates are suggestive of possible resistance mechanisms or secondary targets for a drug combination.
* Dependency-dependency correlations for undruggable genes. For these, the correlates are suggestive of substitute targets, or combinations of targets. However, as combinations are considered the causal interpretation becomes more important.
* Expression-expression correlations. In this case one idea would be to explore the strength and sign of relationships from the data vs. assertions from curated databases/literature, particularly from perturbational experiments.
* Co-expression patterns. Another possibility is to discover patterns of relationships between genes, e.g. that expression of ubiquitin ligases is correlated with the expression of their targets.

Another interesting direction would be to use the information in the INDRA database to discover patterns of mechanistic relationships that predict correlations between dependencies of two genes, or between expression of a gene and dependency of another gene. The simplest possible application of this would be to look at pairwise relationships between two genes (could even extract from SIF), and test to see which types of relationships, and signs of relationships predicted, and amount of evidence predicted a) the sign of a correlation and b) the strength of the correlation.

If the matrix is made complete, i.e., if entries are included for having no known relationship, we may find that having no relationship is not predictive of a lack of a correlation.

Similarly, there will be many mechanistic relationships that yield detectable correlations.

One hypothesis that could be tested is whether downstream siblings were more likely to have large correlations to each other than with their ancestors, highlighting that correlations rarely pick up causal relations in this type of data.

Could do this with graph convolutional nets?

Another application would be to use the data to refine vague relationships extracted from text, or to address issues with assembly, a la distant supervision.

Another direction would be to cluster the correlation graph to find likely intermediates.

Or, to use the redundancy of certain gene families to explain why certain correlations do *not* show up, i.e., the gene knockout has no effect because it has a redundant copy.

# Preliminaries

In [1]:
import pickle
from os import environ
from os.path import join, expanduser
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from indra.databases import hgnc_client
from indra.tools import assemble_corpus as ac

from depmap_analysis.scripts.depmap_script2 import main as run_depmap
from depmap_analysis.scripts.depmap_script2 import mito_file
from depmap_analysis.preprocessing import get_mitocarta_info
from depmap_analysis.network_functions.depmap_network_functions import corr_matrix_to_generator
from depmap_analysis.util.statistics import get_z, get_logp, get_n

from scipy import stats

%matplotlib notebook

MAX_FLOAT = np.finfo(np.float).max
MIN_FLOAT = np.finfo(np.float).min

INFO: [2021-04-30 14:52:17] pybel.config - no configuration found, using default sqlite connection sqlite:////Users/johnbachman/.pybel/pybel_0.14.0_cache.db
INFO: [2021-04-30 14:52:19] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/johnbachman/.indra/bio_ontology/1.12/bio_ontology.pkl

(Background on this error at: http://sqlalche.me/e/e3q8)


## Load transcript and dependency data and compute correlations


Key variables/functions:
* `ccle_df`. CCLE expression data, DataFrame.
* `crispr_df`. DepMap CRISPR gene effect (dependency) data, DataFrame.
* `crispr_corr`
* `crispr_z`
* `rnai_df`. DepMap combined RNAi data, DataFrame.
* `rna_corr`
* `rnai_z`
* `dep_z`
* `dep_df`. CRISPR and RNAi data combined as averaged z-scores, DataFrame.
* `dep_corr`. Correlation matrix between dep-dep, dep-expr, and expr-expr (though expr-expr correlations are only defined in this matrix for the genes also included in the dependency data).

### Working directory

In [2]:
#basedir = 'data/20q4'
basedir_john = '/Users/johnbachman/Dropbox/1johndata/Knowledge File/Biology/Research/Big Mechanism/datasets/depmap_analysis/depmap/21q1'
basedir = environ.get('BASEPATH', basedir_john)
figdir = join(expanduser('~'), 'Dropbox/DARPA projects/papers/INDRA paper 2/figures/figure_panels')

### Mitocarta data

In [3]:
mitocarta = pd.read_excel('data/Human.MitoCarta3.0.xls', sheet_name=1)
mitogenes = list(mitocarta.Symbol.values)

### Cell line metadata

Load the cell line metadata which includes mappings between cell line identifiers used by different datasets.

In [4]:
cell_line_df = pd.read_csv(join(basedir, 'sample_info.csv'))
#cell_line_df = pd.read_csv('data/DepMap-2019q1-celllines_v2.csv')

In [5]:
cell_line_map = cell_line_df[['DepMap_ID', 'CCLE_Name']]
cell_line_map.set_index('CCLE_Name', inplace=True)
cell_line_map.head()

,DepMap_ID
CCLE_Name,
NIHOVCAR3_OVARY,ACH-000001
HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000002
CACO2_LARGE_INTESTINE,ACH-000003
HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000004
HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000005


## Functions for processing RNAi and CRISPR correlations

Functions from Albert to calculate the inverse log CDF of the normal distribution:
(moved to depmap_analysis.util.statistics)

In [6]:
import numpy as np


In [7]:
def get_corr(recalculate, data_df, filename):
    filepath = join(basedir, filename)
    if recalculate:
        %time data_corr = data_df.corr()
        data_corr.to_hdf('%s.h5' % filepath, filename)
    else:
        data_corr = pd.read_hdf('%s.h5' % filepath)
    return data_corr


### RNAi data

Load combined RNAi data, normalize column names, map to DepMap cell line IDs, and drop duplicate columns.

In [8]:
%time rnai_df = pd.read_csv(join(basedir, 'D2_combined_gene_dep_scores.csv'), index_col=0)
rnai_df = rnai_df.transpose()
gene_cols = ['%s' % col.split(' ')[0] for col in rnai_df.columns]
rnai_df.columns = gene_cols
rnai_df = rnai_df.join(cell_line_map)
rnai_df = rnai_df.set_index('DepMap_ID')
# Drop duplicate columns
rnai_df = rnai_df.loc[:,~rnai_df.columns.duplicated()]

CPU times: user 1.97 s, sys: 203 ms, total: 2.17 s
Wall time: 2.18 s


In [9]:
rnai_df.head()

,A1BG,NAT2,ADA,CDH2,AKT3,MED6,NR2E3,NAALAD2,DUXB,PDZK1P1,...,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,KCNE2,DGCR2,CASP8AP2,SCO2
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-001270,NaN,NaN,NaN,-0.194962,-0.256108,-0.174220,-0.140052,NaN,NaN,NaN,...,-0.201644,-0.363670,0.184260,-0.115616,-0.125958,NaN,0.088853,NaN,-0.843295,NaN
ACH-001000,NaN,NaN,NaN,-0.028171,0.100751,-0.456124,-0.174618,NaN,NaN,NaN,...,0.074889,0.152158,0.036011,0.117300,0.101725,NaN,-0.110628,NaN,-0.307031,NaN
ACH-001001,0.146042,0.102854,0.168839,0.063047,-0.008077,-0.214376,-0.153619,0.133830,0.138673,0.030345,...,0.006735,-0.033385,0.197651,-0.016372,0.077486,0.106165,0.057286,0.025596,-0.413669,0.122669
ACH-002319,-0.190388,0.384106,-0.120700,-0.237251,0.060267,-0.338946,-0.057551,0.134511,NaN,0.144463,...,0.209009,-0.156839,-0.155837,-0.001141,NaN,0.227968,0.028095,-0.080611,-1.849696,-0.078856
ACH-001827,0.907063,0.403192,0.004394,-0.017059,-0.094749,-0.328074,-0.089573,0.362029,NaN,-0.098161,...,-0.137465,-1.037848,-0.261262,-0.228016,NaN,0.088744,0.159467,0.014071,-0.414154,0.032661


Calculate RNAi correlations:

In [10]:
recalculate = False

rnai_corr = get_corr(recalculate, rnai_df, 'rnai_correlations')

INFO: [2021-04-30 15:17:18] numexpr.utils - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO: [2021-04-30 15:17:18] numexpr.utils - NumExpr defaulting to 8 threads.


Calculate RNAi sample sizes (number of non-missing values in pairs of columns):

In [11]:
rnai_corr.head()

,A1BG,NAT2,ADA,CDH2,AKT3,MED6,NR2E3,NAALAD2,DUXB,PDZK1P1,...,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,KCNE2,DGCR2,CASP8AP2,SCO2
A1BG,1.000000,-0.053170,-0.057689,0.053209,-0.019174,0.000930,0.089174,0.053059,0.036935,-0.002463,...,0.119704,-0.146314,-0.024557,0.038307,-0.023972,0.017129,0.024641,-0.047776,0.023925,0.100400
NAT2,-0.053170,1.000000,0.051519,0.002823,-0.034266,0.030885,0.005237,-0.039173,-0.017945,-0.106155,...,0.035899,-0.051940,0.008295,0.008854,-0.159054,-0.007945,0.054352,-0.058752,-0.094724,-0.032338
ADA,-0.057689,0.051519,1.000000,-0.028305,-0.038091,0.039840,-0.007825,-0.037555,0.000769,0.016227,...,0.043982,0.010465,0.056808,0.073861,-0.017691,-0.020626,-0.047993,0.106098,0.018621,-0.010055
CDH2,0.053209,0.002823,-0.028305,1.000000,0.050971,-0.066889,-0.003648,0.016123,0.022661,-0.022969,...,-0.037606,-0.131892,0.097606,-0.076574,-0.003035,-0.010893,0.007762,-0.104119,-0.081411,-0.022924
AKT3,-0.019174,-0.034266,-0.038091,0.050971,1.000000,-0.022616,-0.081685,0.052951,0.072240,-0.037919,...,-0.091548,0.027137,0.041509,-0.064823,0.029014,-0.086059,-0.038459,-0.090081,-0.007495,-0.020063


In [12]:
recalculate = False

rnai_n = get_n(recalculate, rnai_df, join(basedir, 'rnai_n'))

8 sec


In [13]:
rnai_n.head()

,A1BG,NAT2,ADA,CDH2,AKT3,MED6,NR2E3,NAALAD2,DUXB,PDZK1P1,...,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,KCNE2,DGCR2,CASP8AP2,SCO2
A1BG,712.0,547.0,547.0,547.0,547.0,547.0,547.0,547.0,501.0,547.0,...,547.0,547.0,547.0,547.0,507.0,547.0,547.0,547.0,547.0,547.0
NAT2,547.0,712.0,547.0,547.0,547.0,547.0,547.0,547.0,501.0,547.0,...,547.0,547.0,547.0,547.0,507.0,547.0,547.0,547.0,547.0,547.0
ADA,547.0,547.0,712.0,547.0,547.0,547.0,547.0,547.0,501.0,547.0,...,547.0,547.0,547.0,547.0,507.0,547.0,547.0,547.0,547.0,547.0
CDH2,547.0,547.0,547.0,712.0,710.0,708.0,708.0,547.0,501.0,547.0,...,710.0,707.0,710.0,710.0,670.0,547.0,708.0,547.0,710.0,547.0
AKT3,547.0,547.0,547.0,710.0,712.0,708.0,708.0,547.0,501.0,547.0,...,710.0,707.0,710.0,710.0,670.0,547.0,708.0,547.0,710.0,547.0


Calculate RNAi p-values:

In [14]:
recalculate = False

rnai_logp = get_logp(recalculate, rnai_n, rnai_corr, join(basedir, 'rnai_logp'))

7.158393144607544 sec


In [15]:
rnai_logp.head()

,A1BG,NAT2,ADA,CDH2,AKT3,MED6,NR2E3,NAALAD2,DUXB,PDZK1P1,...,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,KCNE2,DGCR2,CASP8AP2,SCO2
A1BG,-inf,-1.539955,-1.726559,-1.541511,-0.423808,-0.017465,-3.294917,-1.535465,-0.893049,-0.046918,...,-5.286829,-7.422472,-0.568168,-0.990980,-0.527266,-0.371983,-0.570510,-1.329329,-0.550615,-3.971862
NAT2,-1.539955,-inf,-1.474098,-0.053956,-0.858461,-0.752902,-0.102319,-1.020302,-0.373031,-4.343707,...,-0.911154,-1.490757,-0.166633,-0.178757,-8.035273,-0.159088,-1.587852,-1.771831,-3.621666,-0.797674
ADA,-1.726559,-1.474098,-inf,-0.675583,-0.983717,-1.043109,-0.156524,-0.965797,-0.013796,-0.349666,...,-1.189015,-0.214341,-1.689445,-2.472535,-0.369497,-0.461603,-1.337555,-4.339962,-0.409641,-0.205192
CDH2,-1.541511,-0.053956,-0.675583,-inf,-1.743591,-2.586298,-0.080327,-0.347111,-0.489658,-0.524363,...,-1.148818,-7.733410,-4.682383,-3.185126,-0.064542,-0.223964,-0.178333,-4.210226,-3.503925,-0.523129
AKT3,-0.423808,-0.858461,-0.983717,-1.743591,-inf,-0.601501,-3.514637,-1.531147,-2.241452,-0.977950,...,-4.221340,-0.752339,-1.311764,-2.472910,-0.790988,-3.118282,-1.181474,-3.347257,-0.171997,-0.446845


In [16]:
np.exp(rnai_logp['CASP9']['OSCP1'])

0.0007172501778812698

In [17]:
# Calculate explicitly using scipy.stats.pearsonr
x, y = rnai_df['CASP9'].values, rnai_df['OSCP1'].values
nas = np.logical_or(np.isnan(x), np.isnan(y))
corr = stats.pearsonr(x[~nas], y[~nas])
corr

(-0.18179857862062887, 0.0007172501778812697)

Calculate RNAi z-scores:

In [18]:
recalculate = True

rnai_z = get_z(recalculate, rnai_logp, rnai_corr, join(basedir, 'rnai_z_log'))
#rnai_z_ppf = get_z(recalculate, rnai_logp, rnai_corr, 'rnai_z')

93.67246508598328 sec


In [19]:
rnai_z.head()

,A1BG,NAT2,ADA,CDH2,AKT3,MED6,NR2E3,NAALAD2,DUXB,PDZK1P1,...,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,KCNE2,DGCR2,CASP8AP2,SCO2
A1BG,inf,-1.241582,-1.347263,1.242485,-0.447451,0.021701,2.084980,1.238973,0.824940,-0.057476,...,2.803331,-3.432670,-0.573122,0.894204,-0.538522,0.399715,0.575080,-1.115457,0.558365,2.348715
NAT2,-1.241582,inf,1.202977,0.065879,-0.799822,0.720857,0.122198,-0.914440,-0.400696,-2.484090,...,0.837948,-1.212811,0.193573,0.206617,-3.595446,-0.185393,1.269216,-1.372135,-2.215320,-0.754789
ADA,-1.347263,1.202977,inf,-0.660615,-0.889157,0.930026,-0.182601,-0.876645,0.017173,0.378682,...,1.026786,0.244204,1.326658,1.725871,-0.397387,-0.481344,-1.120533,2.482756,0.434550,-0.234636
CDH2,1.242485,0.065879,-0.660615,inf,1.356654,-1.778643,-0.096890,0.376256,0.506030,-0.536038,...,-1.000618,-3.516092,2.602417,-2.039772,-0.078417,-0.254197,0.206163,-2.436187,-2.169047,-0.534980
AKT3,-0.447451,-0.799822,-0.889157,1.356654,inf,-0.600777,-2.173289,1.236460,1.615031,-0.885140,...,-2.440204,0.720425,1.104573,-1.726046,0.749765,-2.011875,-1.021904,-2.106272,-0.199360,-0.468198


In [20]:
del rnai_df, rnai_corr, rnai_n, rnai_logp

Old way, based on population distribution of all z-scores. Why is this wrong?

In [37]:
#rnai_mean = rnai_corr.values.mean()
#rnai_sd = rnai_corr.values.std()
#rnai_z2 = (rnai_corr - rnai_mean) / rnai_sd

### CRISPR Data

Load CRISPR dependency data and normalize column names to gene names.

In [22]:
%time crispr_df = pd.read_csv(join(basedir, 'Achilles_gene_effect.csv'), index_col=0)
gene_cols = ['%s' % col.split(' ')[0] for col in crispr_df.columns]
crispr_df.columns = gene_cols
# Drop any duplicate columns (shouldn't be any for CRISPR, but just in case)
crispr_df = crispr_df.loc[:,~crispr_df.columns.duplicated()]

CPU times: user 11.1 s, sys: 358 ms, total: 11.5 s
Wall time: 11.6 s


In [23]:
crispr_df.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000004,0.164686,0.094628,-0.190746,-0.009197,0.032529,-0.191355,0.355279,-0.442430,0.298153,0.170712,...,-0.125219,-0.463670,0.031849,-0.007470,0.261947,0.237661,-0.420546,0.277953,0.227172,-0.151192
ACH-000005,-0.097058,0.241606,0.193671,0.156428,-0.195506,-0.337316,0.247947,-0.587806,-0.070959,0.004288,...,-0.212524,-0.410596,-0.180655,-0.080401,0.220940,-0.076012,-0.106125,0.068227,0.022568,-0.261572
ACH-000007,0.066212,0.073850,-0.065867,0.156038,0.100958,0.140997,0.066367,-0.472706,-0.008810,0.293417,...,-0.096162,-0.277202,-0.058930,0.108511,0.212320,-0.011173,-0.352653,0.084244,-0.387673,-0.439552
ACH-000009,0.097872,0.000361,-0.055818,0.046221,0.063069,-0.048932,0.070910,-0.640566,0.148335,0.056188,...,-0.293175,-0.259388,-0.095524,0.049911,0.072002,0.011488,-0.596006,0.189440,-0.117177,-0.539860
ACH-000011,0.270232,0.072105,0.014605,0.433243,-0.030273,-0.239080,0.097866,-0.419689,0.138635,0.106322,...,-0.401794,-0.507831,-0.136369,-0.115541,0.274140,0.154517,-0.234775,0.124355,-0.250505,-0.372498


In [24]:
# Filter out this problematic outlier stomach cell line
#crispr_df = crispr_df[crispr_df.index != 'ACH-000167']

Compute CRISPR correlations:

In [25]:
recalculate = False
    
crispr_corr = get_corr(recalculate, crispr_df, 'crispr_correlations')

Calculate CRISPR sample sizes:

In [26]:
recalculate = False

crispr_n = get_n(recalculate, crispr_df, join(basedir, 'crispr_n'))

10 sec


Calculate CRISPR p-values:

In [27]:
recalculate = False

crispr_logp = get_logp(recalculate, crispr_n, crispr_corr, join(basedir, 'crispr_logp'))

8.052492141723633 sec


In [28]:
crispr_logp.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
A1BG,-inf,-2.729089,-3.607518,-1.980992,-0.071521,-2.398956,-0.056143,-0.356653,-0.068348,-2.486415,...,-0.487791,-0.139625,-0.942707,-0.239842,-1.928554,-1.840513,-1.767935,-1.895396,-1.844012,-1.755956
A1CF,-2.729089,-inf,-0.698920,-0.432430,-0.002479,-5.172074,-0.001276,-1.934047,-4.393969,-0.775754,...,-4.550875,-0.210882,-4.739521,-3.710275,-1.775766,-0.140062,-1.245317,-3.089883,-0.548347,-1.226263
A2M,-3.607518,-0.698920,-inf,-4.959590,-0.624346,-2.082085,-0.231472,-1.223824,-0.122721,-2.952393,...,-0.217154,-2.115227,-0.098963,-2.428453,-0.126596,-0.777269,-0.128080,-0.977968,-2.634127,-1.494651
A2ML1,-1.980992,-0.432430,-4.959590,-inf,-2.025148,-0.460935,-0.555886,-1.550197,-0.033417,-0.563286,...,-8.816429,-4.483147,-4.516258,-3.370472,-0.644246,-0.503514,-0.451465,-0.224883,-1.715529,-0.997601
A3GALT2,-0.071521,-0.002479,-0.624346,-2.025148,-inf,-0.256902,-1.151857,-2.246387,-0.305064,-0.162205,...,-1.027283,-0.496897,-0.532855,-0.532775,-11.028198,-1.635245,-1.599105,-0.067629,-0.483409,-2.461702


In [29]:
import mplayer
p = mplayer.Player()
success_file = '/Users/johnbachman/Downloads/success.m4a'
p.loadfile(success_file)

Calculate CRISPR z-scores

In [30]:
recalculate = False

#crispr_z_ppf = get_z(recalculate, crispr_logp, crispr_corr, 'crispr_z')
crispr_z = get_z(recalculate, crispr_logp, crispr_corr, join(basedir, 'crispr_z_log'))

8.676475048065186 sec


Old way, using population distribution of correlation coefficients:

In [ ]:
#crispr_mean = crispr_corr.values.mean()
#crispr_sd = crispr_corr.values.std()
#crispr_z = (crispr_corr - crispr_mean) / crispr_sd

Clear memory:

In [31]:
del crispr_df, crispr_corr, crispr_n, crispr_logp

 ### Combine CRISPR and RNAi Z-scores
 
 Stouffer's formula:

In [32]:
dep_z = (crispr_z + rnai_z) / np.sqrt(2)

How many infinities due to underflow?

In [33]:
(dep_z.abs() == np.inf).sum().sum()

15669

How many NaN values are there in the combined dataset?

In [34]:
pd.isna(dep_z).sum().sum()

145042232

In [35]:
dep_z.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
A1BG,inf,-2.344420,1.215558,-1.562173,NaN,1.092157,0.348988,0.956904,-1.551909,1.723844,...,2.461753,-0.997734,-0.392137,-0.206969,-1.259928,1.499647,NaN,0.794664,0.817246,0.481082
A1CF,-2.344420,inf,0.176680,1.357196,NaN,-2.648383,0.721116,2.584396,-0.877176,2.035648,...,-3.393022,-0.746085,1.304967,2.017047,-0.127186,2.306467,NaN,0.442976,-1.416195,-0.541671
A2M,1.215558,0.176680,inf,0.005903,NaN,-1.946165,1.592105,-1.093477,-1.687519,-0.266744,...,-1.530944,-0.450800,-1.249026,-0.383748,-1.994584,-0.470479,NaN,-0.543282,1.312836,-0.777776
A2ML1,-1.562173,1.357196,0.005903,inf,NaN,0.337678,-0.333557,0.363633,1.377973,1.380592,...,-2.583980,-1.600073,-0.268438,-2.212961,0.705614,0.269420,NaN,-1.010898,-1.821388,-1.744558
A3GALT2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The large number of NaN values are due to columns being present in RNAi or CRISPR but not both. We drop the NaN rows/columns here:

In [36]:
dep_z = dep_z.dropna(axis=0, how='all').dropna(axis=1, how='all')
pd.isna(dep_z).sum().sum()

0

Convert the z-scores back into p-values:

In [41]:
dep_logp = pd.DataFrame(np.log(2) + stats.norm.logcdf(-dep_z.abs()),
                        index=dep_z.columns, columns=dep_z.columns)

In [42]:
dep_logp

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYX,ZZEF1,ZZZ3
A1BG,-inf,-3.960337,-1.495426,-2.134978,-1.291842,-0.318694,-1.082890,-2.114580,-2.468216,-2.178407,...,-1.797799,-4.281204,-1.144420,-0.363905,-0.179086,-1.571683,-2.012112,-0.851419,-0.882402,-0.461308
A1CF,-3.960337,-inf,-0.151102,-1.744577,-4.817401,-0.753241,-4.629978,-0.966556,-3.175200,-0.083723,...,-0.720491,-7.276993,-0.786105,-1.650760,-3.130621,-0.106702,-3.859215,-0.418880,-1.853304,-0.530951
A2M,-1.495426,-0.151102,-inf,-0.004721,-2.963557,-2.194976,-1.293954,-2.391376,-0.236145,-2.723761,...,-0.032754,-2.073195,-0.427505,-1.552796,-0.355012,-3.077196,-0.449397,-0.532840,-1.664749,-0.828507
A2ML1,-2.134978,-1.744577,-0.004721,-inf,-0.307061,-0.302845,-0.333891,-1.782532,-1.787342,-0.086286,...,-1.348286,-4.628771,-2.211079,-0.237797,-3.615617,-0.733077,-0.238757,-1.164542,-2.680222,-2.512542
A4GALT,-1.291842,-4.817401,-2.963557,-0.307061,-inf,-0.066640,-3.306363,-0.627601,-0.913968,-4.724983,...,-1.128864,-0.179719,-0.505921,-0.366647,-1.474962,-1.692142,-1.132102,-1.183917,-2.753599,-0.492305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,-1.571683,-0.106702,-3.077196,-0.733077,-1.692142,-0.430116,-3.692502,-2.104900,-0.053739,-2.269848,...,-2.839801,-0.529897,-6.057934,-1.179455,-0.637011,-inf,-2.175763,-0.234696,-2.445311,-2.157461
ZYG11A,-2.012112,-3.859215,-0.449397,-0.238757,-1.132102,-3.445092,-0.096436,-3.551059,-0.690674,-1.295834,...,-0.902272,-1.502313,-1.426861,-1.817710,-1.882839,-2.175763,-inf,-0.016753,-3.809342,-0.097269
ZYX,-0.851419,-0.418880,-0.532840,-1.164542,-1.183917,-0.483968,-2.431931,-4.389975,-2.138405,-16.669678,...,-10.844030,-7.859672,-1.039342,-0.171372,-7.555381,-0.234696,-0.016753,-inf,-4.705826,-0.999728
ZZEF1,-0.882402,-1.853304,-1.664749,-2.680222,-2.753599,-1.824835,-1.367785,-1.943995,-0.077415,-3.326127,...,-0.688584,-1.011499,-4.596264,-0.400738,-0.427752,-2.445311,-3.809342,-4.705826,-inf,-2.700534


Save the z-scores:

In [44]:
recalculate = False
filename = 'dep_z'
z_filepath = join(basedir, '%s.h5' % filename)
if recalculate:
    dep_z.to_hdf(z_filepath, filename)
else:
    dep_z = pd.read_hdf(z_filepath)

Old way: take the average of CRISPR and RNAi z-scores:

In [242]:
#recalculate = False
#filename = 'dep_z'
#z_filepath = join(basedir, '%s.h5' % filename)
#if recalculate:
#    %time dep_z = (crispr_z + rnai_z) / 2
#    #dep_z = dep_z.dropna(axis=0, how='all').dropna(axis=1, how='all')
#    dep_z.to_hdf(z_filepath, filename)
#else:
#    dep_z = pd.read_hdf(z_filepath)    

Save the p-values:

In [45]:
recalculate = True
filename = 'dep_logp'
logp_filepath = join(basedir, '%s.h5' % filename)
if recalculate:
    dep_logp.to_hdf(logp_filepath, filename)
else:
    dep_logp = pd.read_hdf(z_filepath)

## Get correlations with correction values

A useful function for looking at filtered sets of correlations:

In [48]:
def unstack_corrs(df):
    df_ut = df.where(np.triu(np.ones(df.shape), k=1).astype(np.bool))
    stacked: pd.DataFrame = df_ut.stack(dropna=True)
    return stacked

Specify which dataset of p-values we'll be testing:

In [49]:
#df_logp = crispr_logp
df_logp = dep_logp

Get the number of gene pairs that we'll be testing. First we get the total number of correlations in the upper triangular part of the matrix:

In [50]:
total_comps = np.triu(~df_logp.isna(), k=1).sum()
total_comps

122719611

Then we calculate how many correlations are between strictly mitochondrial genes. Because we won't be considering these correlations, we subtract these from our number of comparisons:

In [51]:
mitogenes_in_df = set(df_logp.columns).intersection(set(mitogenes))
mito_comps = len(mitogenes_in_df)**2
num_comps = total_comps - mito_comps
num_comps

121778711

 Calculate the log p-value threshold of alpha / num_comps:

In [52]:
alpha = 0.05
bc_thresh = np.log(alpha / num_comps)
bc_thresh

-21.61344838498181

Get all correlations with uncorrected p-values below threshold:

In [57]:
sig_no_corr = unstack_corrs(df_logp[df_logp < np.log(alpha)])

In [58]:
sig_no_corr

A1BG    A1CF    -3.960337
        AAMP    -4.451078
        AANAT   -5.925051
        AARS2   -9.828733
        ABCA1   -3.829890
                   ...   
ZWINT   ZZEF1   -4.596264
ZXDA    ZXDB    -9.143097
ZXDB    ZYX     -7.555381
ZYG11A  ZZEF1   -3.809342
ZYX     ZZEF1   -4.705826
Length: 25343861, dtype: float64

Filter out correlations between mitogenes:

In [59]:
def filt_mitocorrs(ser):
    filt_ix = []
    filt_vals = []
    for (ix0, ix1), logp in ser.iteritems():
        if ix0 in mitogenes and ix1 in mitogenes:
            continue
        filt_ix.append((ix0, ix1))
        filt_vals.append(logp)
    index = pd.MultiIndex.from_tuples(filt_ix, names=['geneA', 'geneB'])
    filt_ser = pd.Series(filt_vals, index=index)
    return filt_ser

In [60]:
filt_corrs = filt_mitocorrs(sig_no_corr)

In [62]:
filt_corrs

geneA   geneB
A1BG    A1CF    -3.960337
        AAMP    -4.451078
        AANAT   -5.925051
        AARS2   -9.828733
        ABCA1   -3.829890
                   ...   
ZWINT   ZZEF1   -4.596264
ZXDA    ZXDB    -9.143097
ZXDB    ZYX     -7.555381
ZYG11A  ZZEF1   -3.809342
ZYX     ZZEF1   -4.705826
Length: 25211483, dtype: float64

In [61]:
import mplayer
p = mplayer.Player()
success_file = '/Users/johnbachman/Downloads/success.m4a'
p.loadfile(success_file)

* Pairs at corrected threshold for CRISPR: 168,098
* For RNAi: 142,430
* For combined by Stouffer: ~62,320

### Calculate correction thresholds (Bonferroni, B-H, B-Y)

In [63]:
sig_sorted = filt_corrs.sort_values().to_frame('logp')

Add rank column:

In [64]:
sig_sorted['rank'] = sig_sorted.rank()

Bonferroni correction threshold:

In [66]:
sig_sorted['bc_cutoff'] = bc_thresh

Benjamini-Hochberg critical value:

In [67]:
sig_sorted['bh_crit_val'] = np.log((sig_sorted['rank'] / num_comps) * alpha)

Benjamini-Yekutieli critical value

In [68]:
cm = np.log(num_comps) + np.euler_gamma + (1/(2*num_comps))

In [69]:
sig_sorted['by_crit_val'] = sig_sorted['bh_crit_val'] - np.log(cm)

In [70]:
sig_sorted

,,logp,rank,bc_cutoff,bh_crit_val,by_crit_val
geneA,geneB,,,,,
TSC1,TSC2,-inf,1.0,-21.613448,-21.613448,-24.568095
MDM2,TP53,-585.802559,2.0,-21.613448,-20.920301,-23.874947
TP53,TP53BP1,-511.445963,3.0,-21.613448,-20.514836,-23.469482
AP2M1,AP2S1,-464.574498,4.0,-21.613448,-20.227154,-23.181800
CDKN1A,TP53,-444.180749,5.0,-21.613448,-20.004010,-22.958657
...,...,...,...,...,...,...
GAPT,MAST1,-2.995733,25211479.0,-21.613448,-4.570638,-7.525285
CCDC58,PNMA6A,-2.995733,25211480.0,-21.613448,-4.570638,-7.525285
PTPRN,ZBTB2,-2.995733,25211481.0,-21.613448,-4.570638,-7.525285


Correlations sig by Bonferroni:

In [73]:
sig_sorted[sig_sorted['logp'] < sig_sorted['bc_cutoff']].shape

(86948, 5)

In [74]:
sig_sorted[sig_sorted['logp'] < sig_sorted['bh_crit_val']].shape

(7349009, 5)

In [75]:
sig_sorted[sig_sorted['logp'] < sig_sorted['by_crit_val']].shape

(1279296, 5)

In [77]:
filename = join(basedir, 'dep_stouffer_signif.pkl')
sig_sorted.to_pickle(filename)

### Expression Data

Load the RNA-seq data (in transcripts per million, obtained from the DepMap website at https://depmap.org/portal/download/). Normalize column names (which include both gene names and Ensemble IDs) to only gene names, and then drop duplicate columns, keeping the first instance of the column for the gene name (there are 87 genes associated with more than one Ensemble ID, and these result in duplicate columns after normalizing to gene names).

In [ ]:
%time ccle_df = pd.read_csv(join(basedir, 'CCLE_expression.csv'), index_col=0)

In [ ]:
gene_cols = ['%s' % col.split(' ')[0] for col in ccle_df.columns]
ccle_df.columns = gene_cols
# NOTE: This first way doesn't work! Probably because with duplicate column names a
# separate integer index is created after transposition (?)
#%time ccle_df_drop = ccle_df.T.drop_duplicates(keep='first').T
# See: https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
ccle_df_drop = ccle_df.loc[:,~ccle_df.columns.duplicated()]

Calculate expression correlation matrix (takes about 11 min):

In [ ]:
recalculate = False
filename = 'expr_correlations'
filepath = join(basedir, filename)
if recalculate:
    %time expr_corr = ccle_df.corr()
    expr_corr.to_hdf('%s.h5' % filepath, filename)
else:
    expr_corr = pd.read_hdf('%s.h5' % filepath)    

Filter the expression data to contain only those columns also contained in the CRISPR dependency data (protein coding genes, for the most part).

In [ ]:
crispr_cols = [c for c in crispr_df.columns]
# Create a boolean mask to filter the CCLE expression data columns
ccle_col_mask = np.array(ccle_df_drop.columns.map(lambda x: x in crispr_cols), dtype=bool)
ccle_df_filt = ccle_df_drop[ccle_df_drop.columns[ccle_col_mask]]

Join the two datasets on the cell line index:

In [ ]:
crispr_expr_join_df = crispr_df.join(ccle_df_filt, how='left', lsuffix=' KO', rsuffix=' RNA')

Calculate the correlations (slow) or reload from HDF5.

In [ ]:
recalculate = False
filename = 'crispr_expr_corr'
filepath = join(basedir, filename)
if recalculate:
    %time ce_corr = crispr_expr_join_df.corr()
    ce_corr.to_hdf('%s.h5' % filepath, filename)
else:
    ce_corr = pd.read_hdf('%s.h5' % filepath)

### Load Drug Data

In [ ]:
drug_resp_df = pd.read_csv(join(basedir, 'primary-screen-replicate-collapsed-logfold-change.csv'), index_col=0)
drug_info_df = pd.read_csv(join(basedir, 'primary-screen-replicate-collapsed-treatment-info.csv'), index_col=0)

In [ ]:
drug_resp_df.head()

Join drug sensitivity to expression on the cell line index:

In [ ]:
drug_expr_df = drug_resp_df.join(ccle_df_filt, how='left')

In [ ]:
drug_expr_df.head()

## Define functions for exploring and visualizing the data

Get the sorted list of correlations for a given gene.

In [ ]:
def sort_corrs(corrs, col_name):
    s = corrs[col_name]
    return sorted(list(zip(s.index, s)), key=lambda x: abs(float(x[1])), reverse=True)

Define a function to get the top correlated genes for a given gene.

In [ ]:
def get_expr_corrs(df, geneX):
    mygene_arr = df[geneX].values
    data = df.values
    corrs = np.zeros(data.shape[1])
    for i in range(data.shape[1]):
        vect = data[:,i]
        r = np.corrcoef(mygene_arr, vect)[0, 1]
        corrs[i] = r
    genes = [c for c in df.columns]
    corr_genes = sorted(list(zip(genes, corrs)), reverse=True, key=lambda x: abs(x[1]))
    corr_genes_dict = {k: v for k, v in corr_genes}
    return corr_genes

In [ ]:
def corrs_for_genes(genes, df=None, cutoff=3.0):
    all_corrs = []
    if df is None:
        df = dep_z
    for gene_a in genes:
        try:
            gene_corrs = sort_corrs(df, gene_a)[1:]
        except KeyError:
            continue
        for gene_b, corr_z in gene_corrs:
            if abs(corr_z > cutoff):
                all_corrs.append((gene_a, gene_b, corr_z))
    return all_corrs

In [ ]:
corrs_for_genes(['SLC43A1', 'SLC43A2'], cutoff=1.0)

In [ ]:
sort_corrs(expr_corr, 'LATS1')

Define a function to plot the relationship between the transcription of two genes, along with a linear regression and the residuals.

In [ ]:
def scatter(df, geneX, y_arr, y_label, bins=30):
    plt.figure(figsize=(12,3))
    plt.subplot(1, 3, 1)
    plt.plot(df[geneX], y_arr, linestyle='', marker='.')
    plt.xlabel('%s TPM' % geneX)
    plt.ylabel(y_label)
    r = np.corrcoef(df[geneX], y_arr)[0][1]
    plt.title('Corr: %.3f' % r)
    slope, intercept, r_value, p_value, std_err = \
                    stats.linregress(df[geneX], y_arr)
    yhat = slope * df[geneX] + intercept
    plt.plot(df[geneX], yhat, color='k')
    residuals = y_arr - yhat
    plt.subplot(1, 3, 2)
    plt.hist(residuals, bins=bins)
    plt.title('Residuals')
    plt.subplot(1, 3, 3)
    stats.probplot(residuals, dist='norm', plot=plt)
    plt.title('Quantile-quantile plot vs. normal')
    return residuals

In [ ]:
res1 = scatter(ccle_df, 'AKT1', ccle_df['BAX'], 'AKT1 TPM', bins=20)

In [ ]:
res2 = scatter(ccle_df, 'YAP1', res1, 'Res 1', bins=20)

Next steps:
1. load the INDRA SIF statement file
2. tabulate explanations for correlations;
3. predict correlations from graph structure, edge properties

In [ ]:
sorted(corrs_for_genes(['KIRREL1'], df=expr_corr), 
       key=lambda x: x[2], reverse=True)

# Paper Figures

In [ ]:
opath = '/Users/johnbachman/Dropbox/1johndata/Knowledge File/Biology/Research/Big Mechanism' \
        '/bioexp_paper/output'
prefix = 'fig6_ipynb'

def fig_path(name, fmt):
    return join(opath, f'{prefix}_{name}.{fmt}')

## Figure: Example correlation

In [ ]:
def plot_example(geneA, geneB):
    lw = 0.5
    ms = 1
    fig = plt.figure(figsize=(4, 2), dpi=150)
    
    # CRISPR plot
    plt.subplot(1, 2, 1)
    ax = plt.gca()
    # Axes lines
    ax.axhline(y=0, color='k', linewidth=lw)
    ax.axvline(x=0, color='k', linewidth=lw)
    plt.plot(crispr_df[geneA].values, crispr_df[geneB].values, marker='.', markersize=ms, linestyle='')
    
    # Plot linear regression
    crispr_lr = linregress(crispr_df[geneA].values, crispr_df[geneB].values)
    plt.plot(crispr_df[geneA].values, crispr_df[geneA].values * crispr_lr.slope + crispr_lr.intercept,
             linestyle='-', linewidth=lw, color='black')
    plt.xlabel(f'{geneA} Gene Effect (CERES)')
    plt.ylabel(f'{geneB} Gene Effect (CERES)')
    ax.text(1.4, 1.5, 'rho = %0.3f' % crispr_lr.rvalue, fontsize=pf.fontsize)
    ax.text(1.4, 1.35, 'z = %0.2f' % crispr_z[geneA][geneB], fontsize=pf.fontsize)
    pf.format_axis(ax)

    # RNAi plot
    plt.subplot(1, 2, 2)
    rnai_df_filt = rnai_df[~pd.isnull(rnai_df[geneA]) & ~pd.isnull(rnai_df[geneB])]
    ax = plt.gca()
    ax.axhline(y=0, color='k', linewidth=lw)
    ax.axvline(x=0, color='k', linewidth=lw)
    plt.plot(rnai_df_filt[geneA].values, rnai_df_filt[geneB].values, marker='.', markersize=ms, linestyle='')
    
    # Plot linear regression
    rnai_lr = linregress(rnai_df_filt[geneA].values, rnai_df_filt[geneB].values)
    plt.plot(rnai_df_filt[geneA].values, rnai_df_filt[geneA].values * rnai_lr.slope + rnai_lr.intercept,
             linestyle='-', linewidth=lw, color='black')
    plt.xlabel(f'{geneA} Gene Effect (DEMETER2)')
    plt.ylabel(f'{geneB} Gene Effect (DEMETER2)')
    ax.text(0.7, 0.69, 'rho = %0.3f' % rnai_lr.rvalue, fontsize=pf.fontsize)
    ax.text(0.7, 0.57, 'z = %0.2f' % rnai_z[geneA][geneB], fontsize=pf.fontsize)
    fig.tight_layout()
    pf.format_axis(ax)
    plt.savefig(fig_path(f'{geneA}_{geneB}_corr_plot', 'pdf'))
    
plot_example('CDKN1A', 'CHEK2')

## Figure: Proportion of correlations between mito genes

In [ ]:
mito2_pcts = []
mito1_pcts = []
corr_range = np.linspace(0, 6, 13)

In [ ]:
corr_range

In [ ]:
def get_mito_pairs():
    results = {}
    for ix in range(len(corr_range)):
        mito2_pairs = []
        mito1_pairs = []
        mito0_pairs = []
        corr_lb = corr_range[ix]
        corr_ub = None if (ix + 1) >= len(corr_range) \
                       else corr_range[ix + 1]
        print(f"Getting correlations between {corr_lb} and {corr_ub}")
        if corr_ub is None:
            z_range = dep_z[(dep_z.abs() >= corr_lb)]
        else:
            z_range = dep_z[(dep_z.abs() >= corr_lb) & (dep_z.abs() < corr_ub)]
        pair_iter = corr_matrix_to_generator(z_range, max_pairs=1000000)
        print("Looping correlations")
        for (a, b), zsc in pair_iter:
            # At least 1 mitogene
            if (a in mitogenes) or (b in mitogenes):
                mito1_pairs.append((a, b, zsc))
                # 2 mitogenes
                if (a in mitogenes) and (b in mitogenes):
                    mito2_pairs.append((a, b, zsc))
            # No mitogenes
            else:
                mito0_pairs.append((a, b, zsc))
        results[(corr_lb, corr_ub)] = {'mito0': mito0_pairs,
                                       'mito1': mito1_pairs,
                                       'mito2': mito2_pairs}
    return results

mito_res = get_mito_pairs()

In [238]:
import mplayer
p = mplayer.Player()
success_file = '/Users/johnbachman/Downloads/success.m4a'
p.loadfile(success_file)

In [ ]:
def plot_mito_pcts(mito_res):
    m1_vals = []
    m2_vals = []
    lbs_vals = []
    for corr_bounds, pair_dict in mito_res.items():
        # Use the lower bound
        lbs_vals.append(corr_bounds[0])
        num_mito0 = len(pair_dict['mito0'])
        num_mito1 = len(pair_dict['mito1'])
        num_mito2 = len(pair_dict['mito2'])
        total_corrs = num_mito0 + num_mito1
        m1_vals.append(num_mito1 / total_corrs)
        m2_vals.append(num_mito2 / total_corrs)
        
    plt.figure(figsize=(2, 2), dpi=150)
    plt.plot(lbs_vals, m1_vals, color='b', label='1 mito gene')
    plt.plot(lbs_vals, m2_vals, color='r', label='2 mito genes')
    ax = plt.gca()
    plt.xlabel('Absolute z-score lower bound')
    plt.ylabel('Pct. of correlations')
    plt.xticks(range(8))
    plt.legend(loc='upper left', fontsize=pf.fontsize, frameon=False)
    plt.subplots_adjust(left=0.16, bottom=0.15)
    pf.format_axis(ax)
    plt.savefig(fig_path(f'mito_corr_pcts', 'pdf'))
    plt.savefig(fig_path(f'mito_corr_pcts', 'png'))
    
plot_mito_pcts(mito_res)   

## Hunting for other mitogenes

In [ ]:
def nonmito(cutoff):
    nonmito_corrs = []
    for (corr_lb, corr_ub), pairs_dict in mito_res.items():
        mito1 = set(pairs_dict['mito1'])
        mito2 = set(pairs_dict['mito2'])
        only1 = mito1.difference(mito2)
        for a, b, zsc in only1:
            if abs(zsc) < 2:
                continue
            if a in mitogenes:
                mitogene = a
                nonmitogene = b
            else:
                mitogene = b
                nonmitogene = a
            nonmito_corrs.append((nonmitogene, mitogene, zsc))
    nonmito_df = pd.DataFrame.from_records(nonmito_corrs, columns=['non_mito', 'mito', 'z_score'])
    # Skips correlations involving only mito genes
    # Find correlations with 1 mitogene
    return nonmito_df
nonmito_df3 = nonmito(3.0)
gb = nonmito_df3.groupby('non_mito')
mito_agg_mean = gb.aggregate(lambda x: x.abs().mean()).sort_values(by='z_score', ascending=False)
mito_agg_mean[0:50]

Based on this analysis, MRPL58, and MCL1, TWNK (Twinkle mtDNA helicase), GATB (Glutamyl-TRNA(Gln) Amidotransferase Subunit B, Mitochondrial), MRM2 (Mitochondrial RRNA Methyltransferase 2), should definitely be included in mitocarta.

COL4A3BP is a ceramide transporter/sphingolipid metabolism regulator, linked to SPTLC2 (sphingolipid metabolism, in Mitocarta). possible interesting link to mitos.

WDR44 and ACAP2 both associated with endosomes/endocytic recycling. WDR44 also linked to iron uptake. Possible link to endosome/mitochondria relationship (which also involves iron). Interestingly, WDR44 is negatively correlated with many mitochondria ribosome subunits.

Weirdly, WDR44 is in this dataset because of strong correlation with KIAA0100, which is listed as a mitochondrila gene? And ACAP2 included because linked to RAB35 (apparently mitochondrial).  Need to understand more about this.

Iron transporter: GTPBP8 (ferrous iron transmembrane transporter activity), TFRC

Nucleotide biosynthesis genes: ADSS (purine), GART (purine), PPAT (purine), ADSL (purine), PFAS (purine), UMPS (pyrimidine).

Peroxisome:
* PEX6. Peroxisomal biogenesis. Correlated with ACOX1 and HSD17B4, both in MitoCarta and with peroxisomal function annotations. Binds CERS2 (ceramide synthase), MARCH5.
* PEX5, PEX12.

Other known:
* MCL1

Mysterious:
* LSM12. Binds PBP1 (stress granules, mRNA processing) and ATXN2, which is classified as a mitochondrial gene in Mitocarta (Genecards: "The encoded cytoplasmic protein localizes to the endoplasmic reticulum and plasma membrane, is involved in endocytosis, and modulates mTOR signals, modifying ribosomal translation and mitochondrial function."). In Yeast, binds a bunch of RNA processing and ribosome proteins (https://thebiogrid.org/36554/summary/saccharomyces-cerevisiae/lsm12.html).
* TAX1BP1. Correlated with NCOA4, androgen receptor coactivator classified as mitochondrial?
* ZNF638. Correlated with TNFRSF10B (DR5), CFLAR, CASP8, BCL2L1 (Bcl-XL).

Now let's aggregate by number of correlations (above cutoff) to see what comes up:

In [ ]:
nonmito_df3[nonmito_df3.non_mito == 'TNFRSF10B']

In [ ]:
nonmito_df3 = nonmito(3.0)
gb = nonmito_df3.groupby('non_mito')
mito_agg_mean = gb.aggregate('count').sort_values(by='z_score', ascending=False)
mito_agg_mean[0:50]

Of these, some should be in Mitocarta:
* KIAA0391 ("Mitochondrial Ribonuclease P Catalytic Subunit")
* CLUH (Clustered Mitochondria Homolog)
* MINOS1 (Mitochondrial Contact Site And Cristae Organizing System Subunit 10)

Other:
* GTPBP8 (iron)

In [ ]:
'RHEBL1' in mitogenes

In [ ]:
pd.DataFrame.from_records(nonmito_corrs[0:3], columns=['a', 'b', 'zsc'])

## Figure: For Mitogenes, strong correlations != stronger effects

In [ ]:
crispr_mean = crispr_df.abs().mean(axis=0).sort_values()

In [ ]:
crispr_z_mean = crispr_z.abs().mean(axis=0)

In [ ]:
n_corrs = 50
np.mean(sorted(crispr_z['CHEK2'].abs(), reverse=True)[1:n_corrs+1])

In [ ]:
#crispr_z_mean = crispr_z.apply(lambda x: np.mean(sorted(x.abs(), reverse=True)[1:101]), axis=1)

In [ ]:
crispr_z_mean.CHEK2

In [ ]:
effect_corr = pd.concat([crispr_mean, crispr_z_mean], axis=1, sort=False)

In [ ]:
mito_effect = effect_corr[effect_corr.index.isin(mitogenes)]
nonmito_effect = effect_corr[~effect_corr.index.isin(mitogenes)]

In [ ]:
crispr_mean

In [ ]:
res = nonmito_effect.groupby(pd.qcut(nonmito_effect[0].values, n_corrs)).mean()

In [ ]:
plt.plot(nonmito_effect[0], nonmito_effect[1], linestyle='', marker='.', color='b', alpha=0.5, label='Not in MitoCarta')
plt.plot(mito_effect[0], mito_effect[1], linestyle='', marker='.', color='r', alpha=0.5, label='In MitoCarta')
plt.plot(res[0].values, res[1].values, linewidth=3, color='black')
plt.xlabel('Average CRISPR Gene Effect')
plt.ylabel(f'Avg of top {n_corrs} CRISPR Corrs (z-scores)')
plt.legend()

## Load Solute Carrier Family genes

In [ ]:
slc_df = pd.read_csv('data/slc_genes.csv')

In [ ]:
slc_genes = list(slc_df['Approved symbol'].values)

In [ ]:
#inet_file = '/Users/johnbachman/Dropbox/1johndata/Knowledge File/Biology/Research' \
#            '/Big Mechanism/datasets/depmap_analysis/graphs/2021-01-06/indranet_dir_graph.pkl'
#with open(inet_file, 'rb') as f:
#    inet = pickle.load(f)

In [ ]:
# Load reactome dict
reactome_file = '/Users/johnbachman/Dropbox/1johndata/Knowledge File/Biology/Research/Big Mechanism/' \
                'datasets/depmap_analysis/reactome_pathways.pkl'

apriori_mapping = get_mitocarta_info(mito_file)

with open(reactome_file, 'rb') as f:
    up2path, _, pathid2pathname = pickle.load(f)
    
reactome_dict = {'uniprot_mapping': up2path,
                 'pathid_name_mapping': pathid2pathname}

expl_funcs=['apriori_explained', 'expl_ab', 'expl_ba', 'expl_axb',
            'expl_bxa', 'find_cp', 'get_st', 'common_reactome_paths']

sd_range = (2, None)
dme_file = 'slc_explainer.pkl'

In [ ]:
run_depmap(inet_file, z_filepath, dme_file, 'unsigned', sd_range, subset_list=slc_genes,
           overwrite=True, reactome_path=reactome_file,
           depmap_date='21q1', expl_funcs=expl_funcs, apriori_explained=mito_file, n_chunks=4)

In [ ]:
import mplayer
p = mplayer.Player()
success_file = '/Users/johnbachman/Downloads/success.m4a'
p.loadfile(success_file)

In [ ]:
with open(dme_file, 'rb') as f:
    dme = pickle.load(f)

In [ ]:
dme_pair = dme.stats_df.set_index('pair').drop(['agA', 'agB', 'z_score'], axis=1)
dme_df = dme.expl_df.join(dme_pair, on='pair')

In [ ]:
slc_to_expl = dme_df[(dme_df['z_score'] > 3) &
                     (dme_df['apriori_explained'] == False) &
                     (dme_df['reactome_paths'] == False)]

In [ ]:
#axb_df = dme.get_filtered_triples_df()
dme.get_filtered_triples_df?

In [ ]:
dme_axb = dme.stats_df.set_index('pair').drop(['agA', 'agB', 'z_score', 'agA_ns', 'agA_id', 'agB_ns', 'agB_id'], axis=1)
axb_all = axb_df.join(dme_axb, on='pair')
axb_all['avg_x_corr'] = (axb_all.ax_corr.abs() + axb_all.bx_corr.abs()) / 2

In [ ]:
axb_all

In [ ]:
slc_to_expl = axb_all[(axb_all['z_score'] > 3) &
                         (axb_all['apriori_explained'] == False) &
                         (axb_all['reactome_paths'] == False)]

In [ ]:
slc_to_expl[slc_to_expl['agA'] ==]

In [ ]:
slc_to_expl[slc_to_expl['avg_x_corr'] > 1.5].sort_values('avg_x_corr', ascending=False)

In [ ]:
import mplayer
p = mplayer.Player()
success_file = '/Users/johnbachman/Downloads/success.m4a'
p.loadfile(success_file)

In [ ]:
dme.plot_corr_stats('slc_output', z_corr=dep_z)

In [ ]:
dme.plot_dists('slc_output', z_corr=dep_z)

In [ ]:
import mplayer
p = mplayer.Player()
success_file = '/Users/johnbachman/Downloads/success.m4a'
p.loadfile(success_file)

In [ ]:
gene = 'SLC2A4'
gene_df = slc_to_expl[(slc_to_expl['agA'] == gene) | (slc_to_expl['agB'] == gene)]
expl_set = []
for row in gene_df.itertuples():
    if row.expl_type == 'shared_regulator':
        continue
    elif row.expl_type == 'shared_target':
        expl_data = row.expl_data[2]
    else:
        expl_data = row.expl_data
    for expl in expl_data:
        expl_set.append(expl)
expl_ctr = Counter(expl_set)
expl_ctr = sorted([(k, v) for k, v in expl_ctr.items()], key=lambda x: x[1], reverse=True)

In [ ]:
gene_df[0:50]

## Other

In [ ]:
from indra.util import batch_iter

In [ ]:
goi = 'MDM2'
res = sort_corrs(dep_z, goi)
#for r in res[0:20]:
#    print(r)
expl_pct = []
batch_size = 1000
for batch in batch_iter(res, batch_size):
    counter = 0
    for gene, corr in batch:
        if gene in indra_dg[goi]:
            counter += 1
    pct = 100 * (counter / batch_size)
    expl_pct.append(pct)

In [ ]:
plt.figure()
plt.bar(range(len(expl_pct)), expl_pct)

# Drug-Drug Correlations

In [ ]:
drug2_corrs = drug_resp_df.corr()

In [ ]:
from indra.sources import tas
tp = tas.process_from_web()

In [ ]:
tas_stmts = tp.statements

In [ ]:
# Build up a dictionary of mappings from CHEBI to SMILES and INCHIKEY
import pyobo
chebi_inchikey_property = 'http://purl.obolibrary.org/obo/chebi/inchikey'
chebi_smiles_property = 'http://purl.obolibrary.org/obo/chebi/smiles'
chebi_id_to_smiles = pyobo.get_filtered_properties_mapping('chebi', chebi_smiles_property)
chebi_id_to_inchikey = pyobo.get_filtered_properties_mapping('chebi', chebi_inchikey_property)

In [ ]:
foo = [stmt for stmt in tas_stmts if stmt.agent_list()[0].name == 'vemurafenib']

In [ ]:
foo[0]

In [ ]:
foo[0].subj.db_refs

In [ ]:
chembl_ids = [stmt.agent_list()[0].db_refs.get('CHEMBL') for stmt in tas_stmts]
chembl_ids = list(set([i for i in chembl_ids if i]))

In [ ]:
len(chembl_ids)

In [ ]:
chembl_ids_str = ';'.join(chembl_ids[0:200])

In [ ]:
j = res.json()

In [ ]:
j['molecules'][0]['molecule_chembl_id']

In [ ]:
import requests
from indra.util import batch_iter
batch_size = 200
chembl_url = 'https://www.ebi.ac.uk/chembl/api/data/molecule/set/'
inchi_keys = []
for ix, batch in enumerate(batch_iter(chembl_ids, batch_size)):
    chembl_ids_str = ';'.join(list(batch))
    print(ix)
    res = requests.get(chembl_url + chembl_ids_str, headers={'Accept': 'application/json'})
    if res.status_code != 200:
        print("Error occurred")
        break
    for mol in j['molecules']:
        cid = mol['molecule_chembl_id']
        ik = mol['molecule_structures']['standard_inchi_key']
        inchi_keys.append((cid, ik))

In [ ]:
j = res.json()

In [ ]:
inchi_to_chembl = {t[1]: t[0] for t in inchi_keys}
chembl_to_inchi = {t[0]: t[1] for t in inchi_keys}

In [ ]:
def get_drug_id(name):
    drug_rec = drug_info_df[drug_info_df.name == name]
    return drug_rec.index.to_list()[0]

def get_drug_name(drug_id):
    drug_rec = drug_info_df.loc[drug_id]
    return drug_rec['name']

def get_drug_targets(drug_id):
    drug_rec = drug_info_df.loc[drug_id]
    targets = drug_rec['target']
    if pd.isna(targets):
        return []
    else:
        return [t.strip() for t in targets.split(',')]

In [ ]:
# Dump all smiles strings from the DepMap data
depmap_smiles = set()
for _, smiles_str in drug_info_df.smiles.items():
    if pd.isna(smiles_str):
        continue
    for sm in smiles_str.split(','):
        sm = sm.strip()
        if sm:
            depmap_smiles.add(sm.strip())
            
depmap_smiles = list(demap_smiles)

with open('depmap_smiles.txt', 'wt') as f:
    for sm in depmap_smiles:
        f.write('%s\n' % sm)

In [ ]:
# Convert from SMILES to INCHIKEY using obabel:
# $ obabel -ismi depmap_smiles.txt -oinchikey > depmap_inchikey.txt

In [ ]:
# Now, read the INCHIKEY databack in and build up a mapping dictionary
depmap_inchikey = []
with open('depmap_inchikey.txt', 'rt') as f:
    depmap_inchikey = [l.strip() for l in f.readlines()]
depmap_smiles_to_ik = dict(zip(depmap_smiles, depmap_inchikey))

In [ ]:
import requests

chembl_url = 'https://www.ebi.ac.uk/chembl/api/data/substructure/'
dmik_to_chembl = {}
for ix, ik in enumerate(depmap_smiles_to_ik.values()):
    if ix > 10:
        break
    res = requests.get(chembl_url + ik, headers={'Accept': 'application/json'})
    if res.status_code != 200:
        print("Error occurred")
        break
    if res.json()['molecules']:
        for mol in res.json()['molecules']:
            cid = mol['molecule_chembl_id']
            if ik not in dmik_to_chembl:
                dmik_to_chembl[ik] = [cid]
            else:
                dmik_to_chembl[ik].append(cid)

In [ ]:
dmik_to_chembl

In [ ]:
drug_name = 'vemurafenib'

In [ ]:
drug_info_df[drug_info_df['name'] == 'vemurafenib']

In [ ]:
sm = [s.strip() for s in drug_info_df[drug_info_df['name'] == 'vemurafenib']['smiles'].values[0].split(',')]

In [ ]:
sm

In [ ]:
for s in sm:
    ik = depmap_smiles_to_ik[s]
    print(ik)

In [ ]:
chembl_to_inchi['CHEMBL1229517']

In [ ]:
drug_name = 'abemaciclib'
corrs = sort_corrs(drug2_corrs, get_drug_id(drug_name))
results = []
corr_k = 20
for i, (corr_drug_id, corr_val) in enumerate(corrs):
    if corr_val == 1.0:
        continue
    if i >= corr_k:
        break
    corr_drug_name = get_drug_name(corr_drug_id)
    corr_targets = get_drug_targets(corr_drug_id)
    for t in corr_targets:
        results.append((corr_drug_name, corr_drug_id, corr_val, t))
tgt_df = pd.DataFrame.from_records(results, columns=('drug_name', 'drug_id', 'corr', 'target'))

In [ ]:
tgt_df

In [ ]:
tgt_df

In [ ]:
tgt_corrs = tgt_df.groupby('target')['corr'].sum()
tgt_counts = tgt_df.groupby('target')['drug_id'].count()
tgt_corrs.sort_values(ascending=False, inplace=True)
tgt_counts.sort_values(ascending=False, inplace=True)

In [ ]:
num_tgts = 20
top_corrs = tgt_corrs[0:num_tgts]
labels = top_corrs.index.to_list()
corrs = top_corrs.values

In [ ]:
plt.figure()
plt.bar(range(len(corrs)), corrs, align='center', tick_label=labels)
plt.xticks(rotation='vertical')
plt.subplots_adjust(bottom=0.2)
plt.xlabel('Nominal targets of top %d correlates' % corr_k)
plt.ylabel('Count')
plt.title('Targets of drugs correlated with %s' % drug_name)
plt.show()

In [ ]:
from indra.util import plot_formatting as pf
from scipy.stats import linregress

geneA = 'CDKN1A'
geneB = 'CHEK2'
lw = 0.5
ms = 1
fig = plt.figure(figsize=(4, 2))
# CRISPR plot
plt.subplot(1, 2, 1)
ax = plt.gca()
ax.axhline(y=0, color='k', linewidth=lw)
ax.axvline(x=0, color='k', linewidth=lw)
plt.plot(crispr_df[geneA].values, crispr_df[geneB].values, marker='.', markersize=ms, linestyle='')
# Plot linear regression
crispr_lr = linregress(crispr_df[geneA].values, crispr_df[geneB].values)
plt.plot(crispr_df[geneA].values, crispr_df[geneA].values * crispr_lr.slope + crispr_lr.intercept,
         linestyle='-', linewidth=lw, color='black')
plt.xlabel(f'{geneA} Gene Effect (CERES)')
plt.ylabel(f'{geneB} Gene Effect (CERES)')
ax.text(1.4, 1.5, 'rho = %0.3f' % crispr_lr.rvalue, fontsize=pf.fontsize)
ax.text(1.4, 1.35, 'z = %0.2f' % crispr_z[geneA][geneB], fontsize=pf.fontsize)
pf.format_axis(ax)
# RNAi plot
plt.subplot(1, 2, 2)
rnai_df_filt = rnai_df[~pd.isnull(rnai_df[geneA]) & ~pd.isnull(rnai_df[geneB])]
ax = plt.gca()
ax.axhline(y=0, color='k', linewidth=lw)
ax.axvline(x=0, color='k', linewidth=lw)
plt.plot(rnai_df_filt[geneA].values, rnai_df_filt[geneB].values, marker='.', markersize=ms, linestyle='')
# Plot linear regression
rnai_lr = linregress(rnai_df_filt[geneA].values, rnai_df_filt[geneB].values)
plt.plot(rnai_df_filt[geneA].values, rnai_df_filt[geneA].values * rnai_lr.slope + rnai_lr.intercept,
         linestyle='-', linewidth=lw, color='black')
plt.xlabel(f'{geneA} Gene Effect (DEMETER2)')
plt.ylabel(f'{geneB} Gene Effect (DEMETER2)')
ax.text(0.7, 0.69, 'rho = %0.3f' % rnai_lr.rvalue, fontsize=pf.fontsize)
ax.text(0.7, 0.57, 'z = %0.2f' % rnai_z[geneA][geneB], fontsize=pf.fontsize)

pf.format_axis(ax)
fig.tight_layout()
plt.savefig(join(figdir, f'{geneA}_{geneB}_corr_plot.pdf'))
dep_z[geneA][geneB]

# Deprecated/Old

## Load Cosmic Data

In [ ]:
cos_df = pd.read_csv('data/Census_allThu Jun 13 20_37_30 2019.csv')

In [ ]:
cos_df.head()

In [ ]:
cos_corrs = corrs_for_genes(cos_df['Gene Symbol'])

In [ ]:
dme

In [ ]:
5+5

In [ ]:
slc_df.head()
slc_corrs = corrs_for_genes(slc_df['Approved symbol'])

In [ ]:
slc_df

In [ ]:
mitocarta = pd.read_excel('data/Human.MitoCarta2.0.xls', sheet_name=1)
mitogenes = list(mitocarta.Symbol.values)

In [ ]:
'PNPT1' in mitogenes

In [ ]:
slc_corrs

In [ ]:
slc_corrs_filt = []
slc_corrs_seen = set()
for corr in slc_corrs:
    # Filter out correlations we've already seen
    corr_set = frozenset(corr)
    if corr_set in slc_corrs_seen:
        continue
    else:
        slc_corrs_seen.add(corr_set)
    # Filter out self-correlations
    if corr[0] == corr[1]:
        continue
    # Filter out mitochondrial gene correlations
    elif corr[0] in mitogenes or corr[1] in mitogenes:
        continue
    else:
        slc_corrs_filt.append(corr)

In [ ]:
slc_corrs_filt.sort(key=lambda x: x[2], reverse=True)

In [ ]:
slc_corrs_filt

In [ ]:
len(slc_corrs_filt)

In [ ]:
import csv
with open('slc_corrs_no_mito.csv', 'wt') as f:
    csv_writer = csv.writer(f, delimiter=',')
    csv_writer.writerows(slc_corrs_filt)

In [ ]:
import requests

In [ ]:
query = {'source': None,
         'target': None,
         'stmt_filter': ['conversion', 'fplx'],
         'node_filter': ['hgnc', 'fplx', 'chebi', 'pubchem', 'go', 'mesh'],
         'node_blacklist': [],
         'edge_hash_blacklist': [],
         'path_length': 1,
         'sign': 'no_sign',
         'weighted': False,
         'bsco': 0.,
         'direct_only': False,
         'curated_db_only': False,         
         'fplx_expand': False,
         'simple': False,
         'k_shortest': False}

def get_expl_corrs(gene_pairs):
    no_path = []
    has_2path = []
    has_3path = []
    for ix, (gene_a, gene_b, z_score) in enumerate(gene_pairs):
        for source, target in ((gene_a, gene_b), (gene_b, gene_a)):
            print("%d getting paths for %s, %s" % (ix, source, target))
            query['source'] = source
            query['target'] = target
            query['path_length'] = 1
            res = requests.post('http://127.0.0.1:5000/query/submit', json=query).json()
            if '2' in res['result']['paths_by_node_count']:
                has_2path.append((source, target, z_score))
            else:
                query['path_length'] = 2
                res = requests.post('http://127.0.0.1:5000/query/submit', json=query).json()
                if '3' in res['result']['paths_by_node_count']:
                    has_3path.append((source, target, z_score))                
                else:
                    no_path.append((source, target, z_score))
    return {'no_path': no_path, 'has_2path': has_2path, 'has_3path': has_3path}

In [ ]:
paths = get_expl_corrs(sorted(slc_corrs, key=lambda x: x[2], reverse=True))

In [ ]:
paths['has_2path']

In [ ]:
paths['has_3path']

In [ ]:
paths['no_path']

In [ ]:
query['source'] = 'GRSF1'
query['target'] = 'SLC30A9'
query['path_length'] = 2
res = requests.post('http://127.0.0.1:5000/query/submit', json=query).json()

In [ ]:
res

## Load INDRA SIF DB

In [ ]:
stmts_df = pd.read_csv('data/stmts_by_pair_type.csv')
# Filter to HGNC only
stmts_df = stmts_df[(stmts_df['agA_ns'] == 'HGNC') & (stmts_df['agB_ns'] == 'HGNC')]
# Remove namespace and identifier columns, leaving only name
stmts_df = stmts_df[['agA_name', 'agB_name', 'stmt_type', 'evidence_count']]
# Remove self-edges
stmts_df = stmts_df[stmts_df['agA_name'] != stmts_df['agB_name']]
# Useful bits of Pandas to know, ultimately not used here:
# pd.crosstab([foo['agA_name'], foo['agB_name']], foo['stmt_type'], foo['evidence_count'],
#             aggfunc='sum', dropna=False).fillna(0)
# baz = bar[bar.apply(lambda x: x.name[0] != x.name[1], axis=1)]

In [ ]:
x_dict = {}
for row in stmts_df.values:
    try:
        raw_corr = dep_z[row[0]][row[1]]
        """
        if int(raw_corr) == 0:
            corr = 0
        elif int(raw_corr) < 0:
            corr = -1
        elif int(raw_corr) > 0:
            corr = 1
        """
        corr = raw_corr
    except KeyError:
        continue
    stmt_type = row[2]
    gene_pair = (row[0], row[1])
    if gene_pair not in x_dict:
        x_dict[gene_pair] = {'Corr': corr}
    x_dict[gene_pair][stmt_type] = row[3]
features = ['Activation', 'Inhibition', 'Corr']
all_data = np.zeros((len(x_dict), len(features)))
for row_ix, feat_dict in enumerate(x_dict.values()):
    for feat_name, feat_val in feat_dict.items():
        if feat_name not in features:
            continue
        feat_ix = features.index(feat_name)
        all_data[row_ix, feat_ix] = feat_val

In [ ]:
"""
# Shuffle the matrix in place
np.random.shuffle(all_data)
# Balance the data
ctr = Counter(all_data[:, -1])
classes = sorted([(k, v) for k,  v in ctr.items()], key=lambda x: x[0])
# Get the class with the fewest members
min_class_count = min([t[1] for t in classes])
min_class_count
data_bal = np.zeros((min_class_count * len(classes), len(features)))
row_ix = 0
for one_class, count in classes:
    num_class_rows = 0
    for ix in range(all_data.shape[0]):
        if all_data[ix, -1] == one_class:
            data_bal[row_ix] = all_data[ix,:]
            row_ix += 1
            num_class_rows += 1
        if num_class_rows >= min_class_count:
            print("Finished class %s, at rows %s" % (one_class, row_ix))
            break

"""

In [ ]:
# Divide training and test
np.random.shuffle(all_data)
partition_ix = int(len(all_data) * 0.8)
train_data = all_data[0:partition_ix, :]
test_data = all_data[partition_ix:, :]
x_train = train_data[:,:-1]
y_train = train_data[:, -1]
x_test = test_data[:, :-1]
y_test = test_data[:, -1]

## Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train, y_train)
nb.score(x_test, y_test)

In [ ]:
yctr = Counter(y_train)
yclasses = sorted([(k, v) for k,  v in yctr.items()], key=lambda x: x[0])

In [ ]:
[features + ['Count']] + list(zip(nb.coef_, classes))

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score

lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)
lr.coef_

In [ ]:
y_pred = lr.predict(x_train)
explained_variance_score(y_train, y_pred)

In [ ]:
y_mean = np.full(y_train.shape, y_train.mean())
explained_variance_score(y_train, y_mean)

In [ ]:
y_pred_test = lr.predict(x_test)
explained_variance_score(y_test, y_pred_test)

## Load INDRA Statements (deprecated)

In [ ]:
reload = False
if reload:
    inc_stmts = ac.load_statements('increase_amt.pkl')
    dec_stmts = ac.load_statements('decrease_amt.pkl')
    stmts = inc_stmts + dec_stmts
    stmts = ac.map_grounding(stmts)
    stmts = [s for s in stmts if s.subj and s.subj.db_refs.get('HGNC')
                             and s.obj and s.obj.db_refs.get('HGNC')]
    stmts = ac.run_preassembly(stmts)
    ac.dump_statements(stmts, 'assembled_stmts.pkl')
else:
    stmts = ac.load_statements('assembled_stmts.pkl')

In [ ]:
stmts_by_obj = defaultdict(list)
for s in stmts:
    gene_id = s.obj.db_refs['HGNC']
    gene_name = hgnc_client.get_hgnc_name(gene_id)
    stmts_by_obj[gene_name].append(s)

In [ ]:
stmt_counts = []
for obj, stmts in stmts_by_obj.items():
    stmt_counts.append((obj, len(stmts)))
stmt_counts.sort(key=lambda x: x[1], reverse=True)

In [ ]:
stmt_counts[0:10]

In [ ]:
gene = 'BCL2'
rank = 0
stmts_by_obj[gene].sort(key=lambda s: s.belief, reverse=True)
print(stmts_by_obj[gene][rank], '\n')
print('\n'.join(['%d: %s\n' % (i, str((e.source_api, e.text)))
                 for i, e in enumerate(stmts_by_obj[gene][rank].evidence)]))

Checking the number of infinities in the z scores:

In [126]:
(crispr_z.abs() == np.inf).sum().sum()

18121

In [127]:
(crispr_z_ppf.abs() == np.inf).sum().sum()

129863

In [129]:
(rnai_z.abs() == np.inf).sum().sum()

17313

In [128]:
(rnai_z_ppf.abs() == np.inf).sum().sum()

66249

### Cases of strongly conflicting sign of correlation

Here are some interesting cases where nans appear in the dataset because the z-scores from small pvalues yield opposing +/- inf in CRISPR vs. RNAi. Most of these appear to be members of families.

In [136]:
val = 123456
dep_z_copy = dep_z.copy()
# Replace NaNs with an arbitrary value so we can get them later
dep_z_copy[pd.isna(dep_z_copy)] = val
confs = dep_z_copy[dep_z_copy == val].where(
                np.triu(np.ones(dep_z_copy.shape), k=1).astype(np.bool))
stacked: pd.DataFrame = confs.stack(dropna=True)
stacked

Series([], dtype: float64)

Here's another weird case, where PSMA2 and RPL18 have such similarly (but not identical) extreme p-values that they are converted to identical z-scores and cancel out to 0 when added:

In [137]:
dep_z_copy = dep_z.copy()
# Replace NaNs with an arbitrary value so we can get them later
confs = dep_z_copy[dep_z_copy == 0].where(
                np.triu(np.ones(dep_z_copy.shape), k=1).astype(np.bool))
stacked: pd.DataFrame = confs.stack(dropna=True)
stacked

Series([], dtype: float64)

In [138]:
crispr_logp['PSMA2']['RPL18']

-36.60307091042653

In [139]:
crispr_z['PSMA2']['RPL18']

8.276444151723624

In [140]:
rnai_logp['PSMA2']['RPL18']

NameError: name 'rnai_logp' is not defined

In [ ]:
rnai_z['PSMA2']['RPL18']

In [141]:
del dep_z_copy